
# Limpieza y transformación de datos (Notebook resuelto)
Dataset: `dataset_sucio.csv` (nulos, duplicados, outliers, fechas en texto y variables categóricas).  
Objetivo: preparar los datos para modelado sin entrar aún en EDA completo.


## 1. Carga del dataset y exploración inicial

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option("display.max_columns", None)

df = pd.read_csv("dataset_sucio.csv")
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='all').T

In [ ]:
df.departamento.unique()

In [ ]:
df.departamento.value_counts()

## 2. Detección y eliminación de duplicados

In [ ]:
df.duplicated().sum(

In [ ]:
duplicados_absolutos = df.duplicated().sum() #numero de duplicados que tengo
duplicados_absolutos

In [ ]:
df.duplicated(subset=["id"]).sum() #cuantos tienen mismo id

In [ ]:
df.duplicated(subset=["apellido"]).sum() #cuantos tienen mismo apellido

In [ ]:
# FILAS DUPLICADAS
df[df.duplicated(keep='last')]

In [ ]:
lucia= df[(df.nombre=='Lucía')]
lucia[lucia.apellido=='Martín']

In [ ]:
df[df.id==190]

In [ ]:
df[df.duplicated()] # veo filas duplicadas 

In [ ]:
df_limpio = df[~df.duplicated()]
len(df_limpio)

In [ ]:
# BORRADO DE FILAS DUPLICADAS 
df.drop_duplicates() # Los duplicados de filas enteras

In [ ]:
# Eliminamos duplicados completos primero
df = df.drop_duplicates().copy()
# También nos aseguramos que cada 'id' sea único
df = df.drop_duplicates(subset=["id"]).copy()
df.shape

## 3. Detección y tratamiento de nulos

##### Podeis usar tanto .isnull() como .isna()

In [ ]:
df.isna()

In [ ]:
df.isnull().sum()

In [ ]:
df_sin_nulos = df[~df.fecha_contrato.isna()]
df_sin_nulos.shape

In [ ]:
df_sin_nulos.info()

In [ ]:

# Ejemplo: imputar nulos en 'ciudad' con 'Desconocido'
df["ciudad"] = df["ciudad"].fillna("Desconocido")

# Ejemplo: eliminar filas con fecha_contrato nula
df = df.dropna(subset=["fecha_contrato"])
df.isnull().sum()


In [ ]:
# IMPUTAR CONTINUA 
# - media, mediana --> crea sesgos
# - valor anterior o posterior, media del valor anterior y posterior --> acciones y series temporales

## 4. Detección y manejo de outliers

In [ ]:
# Salario: revisar valores extremos
df["salario"].describe()

In [ ]:
# Podemos recortar valores de salario demasiado altos
#q_low, q_high = df["salario"].quantile([0.2, 0.8])
#df["salario"] = df["salario"].clip(lower=q_low, upper=q_high)
#df["salario"].describe()


In [ ]:
df.head()

In [ ]:
# CAMBIANDO VALORES!!!!!!! Manipulando los datos!!!!

##### MEJOR OPCION - quitar las filas con valores outliers IQR

In [ ]:
df.describe()

In [ ]:
# Calcular Q1 (25%) y Q3 (75%)
Q1 = df['salario'].quantile(0.25)
print('Q1 = ', Q1)
Q3 = df['salario'].quantile(0.75)
print('Q3 = ', Q3)

# Calcular IQR
IQR = Q3 - Q1
print('IQR = ', IQR)

# Definir límites inferior y superior
limite_inferior = Q1 - 0.5 * IQR
limite_superior = Q3 + 2.5 * IQR
print(limite_inferior)
print(limite_superior)

In [ ]:
1.5*IQR

In [ ]:
df['salario'].min()

In [ ]:
df[df['salario'] > 62500]

In [ ]:
# Filtrar los datos dentro de los límites
df_sin_outliers = df[(df['salario'] >= limite_inferior) & (df['salario'] <= limite_superior)]
len(df_sin_outliers)

## 5. Transformaciones de columnas

In [ ]:
df.info()

In [ ]:

# Convertir columna fecha_contrato a datetime (invalid -> NaT)
df["fecha_contrato"] = pd.to_datetime(df["fecha_contrato"], errors="coerce")
df[["fecha_contrato"]].head()


In [ ]:
# Cambiar tipo de 'departamento' a categoría
df["departamento"] = df["departamento"].astype("category")
df.dtypes


In [ ]:

# Ejemplo extra 1: normalizar nombres a minúsculas
df["nombre"] = df["nombre"].str.lower()

df["apellido"] = df["apellido"].str.lower()

df[["nombre","apellido"]].head()

In [ ]:
# QUITAR TODOS LAS TILDES DE NOMBRE Y APELLIDO

df['nombre'] = df['nombre'].apply(lambda x: x.replace('í','i').replace('ó','o').replace('é','e'))
df['apellido'] = df['apellido'].apply(lambda x: x.replace('í','i').replace('ó','o').replace('é','e'))

df[['nombre','apellido']].head()

In [ ]:
# Ejemplo extra 2: crear columna año de la fecha
df["anio_contrato"] = df["fecha_contrato"].dt.year
df[["fecha_contrato","anio_contrato"]].head()

In [ ]:
# ESTO DARA ERROR PORQUE HAY NULOS!!! Primero quitad o imputad los nulos
df["anio_contrato"] = df["anio_contrato"].astype(int)

## 6. Binning de variables

In [ ]:
df.edad.describe()

In [ ]:

# Con pd.cut: agrupar edades
df["grupo_edad"] = pd.cut(df["edad"], bins=[17,29,49,69], labels=["joven","adulto","mayor"])
df[["edad","grupo_edad"]].head()


In [ ]:
df.grupo_edad.value_counts()

In [ ]:
# Con apply + lambda
df["rango_salario"] = df["salario"].apply(lambda x: "alto" if x>30000 else ("medio" if x>25000 else 'bajo'))
df[["salario","rango_salario"]].head()

## 7. Encoding de variables categóricas

In [ ]:
df.departamento.unique()

In [ ]:
### ESTO NO ES MUY BUENA PRACTICA SI NO ES ORDINAL
# Label encoding sencillo
df["departamento_code"] = df["departamento"].cat.codes
df[["departamento","departamento_code"]].head()


### ONE-HOT ENCODING

In [ ]:
onehot_df = pd.get_dummies(df['departamento'])*1
onehot_df

In [ ]:
# One-hot encoding
df_encoded = pd.get_dummies(df, columns=["ciudad"]) # NO HACER SI HAY MUCHAS CATEGORIAS!
df_encoded.head()


In [ ]:
df_encoded.drop(columns=['id','nombre','apellido'],inplace=True)

In [ ]:
df_encoded.head()

## 8. Feature Engineering = Creatividad

In [ ]:
# Crear margen = ventas - costes
df["margen"] = df["ventas"] - df["costes"]
df.head()

In [ ]:
# Crear beneficio relativo
df["beneficio_pct"] = (df["margen"] / df["ventas"]).round(2)

In [ ]:
df.head()

In [ ]:
# Crear columna nombre_completo
df["nombre_completo"] = df["nombre"] + " " + df["apellido"]
df.head()

In [ ]:
# Crear variable booleana
df["es_joven"] = df["edad"] < 30
df["es_joven"] = df["es_joven"]*1  # Lo que es False se pone 0 y lo que es True se pone 1
df.head()

In [ ]:
# Extraer mes del contrato
df["fecha_contrato"] = df["fecha_contrato"]
df["mes_contrato"] = df["fecha_contrato"].dt.month
df.head()
